In [1]:
import sys
import os
import random
# Counter
from collections import Counter
# Package
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
from somhos.methods.useful import save_pickle, load_pickle, wordvectors_centroid
from somhos.config.paths import *

Default path

In [2]:
prefix_path = "../../"
data_path = get_relative_path(prefix_path, V9_PATH)
os.path.exists(data_path)

True

Load samples

In [3]:
# Load sample A
docs_sample_a_path = get_relative_path(data_path, DOCS_SAMPLE_A_SUFFIX)
docs_sample_a = load_pickle(docs_sample_a_path)
# Load sample B
docs_sample_b_path = get_relative_path(data_path, DOCS_SAMPLE_B_SUFFIX)
docs_sample_b = load_pickle(docs_sample_b_path)

print("Samples size: (%d, %d)" % (len(docs_sample_a), len(docs_sample_b)))

Samples size: (1000, 1000)


Load samples content

In [4]:
docs_samples_content_path = get_relative_path(data_path, DOCS_SAMPLES_CONTENT)
samples_content = load_pickle(docs_samples_content_path)
print("Docs: %d" % len(samples_content))

Docs: 2000


In [5]:
from itertools import product
import numpy as np
from scipy.spatial.distance import cosine as cosine_distance

Load counts documents by keyprase

In [6]:
docs_samples_kps_doc_count_path = get_relative_path(data_path, DOCS_SAMPLES_KPS_DOC_COUNT)
samples_kps_in_docs_count = load_pickle(docs_samples_kps_doc_count_path)

Measure $idf(t, D) = log\frac{N}{\vert \left\{ d\in D:t\in d \right\} \vert}$

In [7]:
n_docs = len(docs_sample_a) + len(docs_sample_b)
t_idf = {t: np.log(n_docs/v) for t, v in samples_kps_in_docs_count.items()}

In [8]:
sample_a_indices = range(0, len(docs_sample_a))
sample_b_indices = range(0, len(docs_sample_b))

def get_content_for(index, samples_content, field='kps-normalized'):
    return samples_content[index][field]

def tf_doc(terms):
    tf = Counter(terms)
    cnt = sum(tf.values())
    return {k: v/cnt for k, v in tf.items()}

m_measure_sim = np.zeros([len(docs_sample_a), len(docs_sample_b)]) 
for i, j in product(sample_a_indices, sample_a_indices):
    # Term frequencies by document
    tf_a = tf_doc(get_content_for(docs_sample_a[i], samples_content))
    tf_b = tf_doc(get_content_for(docs_sample_b[j], samples_content))
    # Term list
    t_list = list(set(tf_a) | set(tf_b))
    # TF-IDF
    tf_idf_a = np.zeros([len(t_list),])
    tf_idf_b = np.zeros([len(t_list),])
    for pos, t in enumerate(t_list):
        if t in tf_a:
            tf_idf_a[pos] = tf_a[t] * t_idf[t]
        if t in tf_b:
            tf_idf_b[pos] = tf_b[t] * t_idf[t]
    # Measure
    measure_sim = 1.0 - cosine_distance(tf_idf_a, tf_idf_b)
    m_measure_sim[i, j] = measure_sim

print(m_measure_sim)

[[0.         0.         0.         ... 0.01244969 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.01317411 0.         0.00451545]
 ...
 [0.         0.         0.         ... 0.         0.00371959 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00627728 0.         0.         ... 0.         0.         0.        ]]


In [9]:
docs_samples_measure_sim_path = get_relative_path(data_path, DOCS_SAMPLES_TF_IDF_KPS)
save_pickle(m_measure_sim, docs_samples_measure_sim_path)

In [10]:
n_singularvalues = 1
U, s, V = np.linalg.svd(m_measure_sim, full_matrices=False)
D = np.diag(s[:n_singularvalues])
m_udv_measure_sim = np.dot(U[:,:n_singularvalues], np.dot(D, V[:n_singularvalues,:]))
print(m_udv_measure_sim)

[[4.73592033e-04 2.61405156e-03 6.42965835e-04 ... 1.01932404e-03
  1.88485822e-04 5.07716514e-04]
 [7.72944305e-04 4.26636455e-03 1.04937741e-03 ... 1.66362746e-03
  3.07625621e-04 8.28638492e-04]
 [1.35137968e-03 7.45911228e-03 1.83468239e-03 ... 2.90860846e-03
  5.37838249e-04 1.44875279e-03]
 ...
 [3.77351905e-04 2.08284191e-03 5.12306722e-04 ... 8.12183992e-04
  1.50183025e-04 4.04541842e-04]
 [2.41010698e-04 1.33028925e-03 3.27204922e-04 ... 5.18733384e-04
  9.59203205e-05 2.58376625e-04]
 [1.54619475e-03 8.53441887e-03 2.09917044e-03 ... 3.32791383e-03
  6.15373081e-04 1.65760518e-03]]


In [11]:
docs_samples_measure_sim_udv_path = get_relative_path(data_path, DOCS_SAMPLES_TF_IDF_UDV_KPS)
save_pickle(m_udv_measure_sim, docs_samples_measure_sim_udv_path)